# Test Script: ZIP Code Geospatial Imputation

**Objective:** This script validates the accuracy of our custom-built Massachusetts ZIP code shapefile (`massachusetts_zips.shp`) used for imputing missing zip codes.

**Methodology:**
* **Shapefile Creation:** The `massachusetts_zips.shp` was created by the `scripts/02_filter_shapefile.py` script. This script takes the national ZCTA (ZIP Code Tabulation Area) shapefile from the US Census Bureau (`tl_2024_us_zcta520`) and filters it to include only the ZIP codes that intersect with the Massachusetts state boundary (`tl_2024_us_state`), creating a much smaller and more efficient file for our analysis.
* **Validation Process:** The script defines several well-known Boston landmarks with their correct ZIP codes. It then performs a **spatial join** (a point-in-polygon operation) to see if the coordinates of these landmarks fall within the correct ZIP code polygon in our shapefile.

**Expected Outcome:** A successful test will show a 100% match between the "Expected ZIP" and the "Found ZIP," confirming our geospatial data is reliable.

In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# --- 1. Load the processed Massachusetts shapefile ---
shapefile_path = '../data/processed/mass_shapefile/massachusetts_zips.shp'
print(f"Loading shapefile from: {shapefile_path}")
ma_zips_gdf = gpd.read_file(shapefile_path)

# --- 2. Define 10 test points with their known correct ZIP codes ---
test_points = {
    'Location': [
        'Fenway Park', 'Faneuil Hall', 'Logan Airport (Terminal E)', 'Franklin Park Zoo',
        '341 Harrison Ave', 'Museum of Science', 'Isabella Stewart Gardner Museum',
        'Boston Public Library', 'New England Aquarium', 'USS Constitution'
    ],
    'Latitude': [
        42.3467, 42.3601, 42.3649, 42.3026, 42.3452, 42.3673, 42.3382,
        42.3496, 42.3592, 42.3727
    ],
    'Longitude': [
        -71.0972, -71.0545, -71.0221, -71.0890, -71.0638, -71.0711, -71.0991,
        -71.0777, -71.0505, -71.0560
    ],
    'Expected ZIP': [
        '02215', '02109', '02128', '02121', '02118', '02114', '02115',
        '02116', '02110', '02129'
    ]
}
test_df = pd.DataFrame(test_points)

# --- 3. Convert the test data into a GeoDataFrame ---
geometry = [Point(xy) for xy in zip(test_df['Longitude'], test_df['Latitude'])]
test_gdf = gpd.GeoDataFrame(test_df, geometry=geometry, crs="EPSG:4326")
test_gdf = test_gdf.to_crs(ma_zips_gdf.crs)

# --- 5. Perform the spatial join ---
results_gdf = gpd.sjoin(test_gdf, ma_zips_gdf[['ZCTA5CE20', 'geometry']], how="left", predicate="within")

# --- 6. Compare the results and print the report ---
results_gdf.rename(columns={'ZCTA5CE20': 'Found ZIP'}, inplace=True)
results_gdf['Match'] = results_gdf['Expected ZIP'] == results_gdf['Found ZIP']

print("\n--- TEST REPORT --- ✅")
results_gdf[['Location', 'Expected ZIP', 'Found ZIP', 'Match']]

Loading shapefile from: ../data/processed/mass_shapefile/massachusetts_zips.shp

--- TEST REPORT --- ✅


,Location,Expected ZIP,Found ZIP,Match
0,Fenway Park,02215,02215,True
1,Faneuil Hall,02109,02109,True
2,Logan Airport (Terminal E),02128,02128,True
3,Franklin Park Zoo,02121,02121,True
4,341 Harrison Ave,02118,02118,True
5,Museum of Science,02114,02114,True
6,Isabella Stewart Gardner Museum,02115,02115,True
7,Boston Public Library,02116,02116,True
8,New England Aquarium,02110,02110,True
9,USS Constitution,02129,02129,True
